In [34]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 일단 개수를 줄여서 돌려보자.
## train_x 는 series of sentences
## train_y 는 serise of labels

In [36]:
# train_size  = 15000  # size 상수
const_tfidf = 1000   #구할 tfidf상위 단어의 개수
const_kai   = 1000   #구할 카이제곱상위 단어의 개수
# test_size   = int(train_size/3)
# train_x, train_y = train_df["document"][:train_size], train_df["label"][:train_size]
# test_x, test_y = test_df["document"][:test_size], test_df["label"][:test_size]

# 긍/ 부정 문서 카이제곱 상위단어 구하기

In [41]:
def wordCount(data, w):
    cnt = 0;
    for s in data:
        if w in s:
            cnt += 1
    return cnt;

def getKai(data, w, c):
    A = wordCount(data[c], w)
    B = wordCount(data[(c+1)%2], w)
    C = len(data[c]) - A
    D = len(data[(c+1)%2]) - B
    # 분모가 0이면 0을 리턴함.
    if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
        return 0
    return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))

In [42]:
import pickle
with open('X_sample.pickle', 'rb') as f:
    train_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('y_sample.pickle', 'rb') as f:
    train_y  = pickle.load(f) # 단 한줄씩 읽어옴

In [43]:
# tfidf_dict = tfidf_dict[:const_tfidf]
train_x = list(train_x)
train_y = list(train_y)

In [44]:
words = []
for s in doc:
    for w in s:
        words.append(w)
words = list(set(words))
data = [list(train_x[train_y == 0]), list(train_x[train_y == 1])]

In [45]:
kai_dict = []
for w in tqdm(words):
    kai_dict.append(getKai(data, w, 0))
kaiRate = pd.DataFrame()
kaiRate['words'] = words
kaiRate.index = words
kaiRate['word_kai'] = kai_dict
kaiRate = kaiRate.sort_values(by=['word_kai'],ascending=False)

100%|██████████| 11060/11060 [00:00<00:00, 777608.33it/s]


In [46]:
kaiRate = kaiRate[:const_kai]

In [47]:
kaiRate = list(kaiRate["words"])

In [48]:
from scipy import sparse
def create_vec_count(x_data, voca):
    vec = np.ndarray((len(x_data), len(voca)))
    for i, s in enumerate(tqdm(x_data)): 
        for j, w in enumerate(voca):
            vec[i][j] = s.count(w)
    return sparse.csr_matrix(vec)
#     return vec

# 원핫

In [49]:
def create_vec_sparse(x_data, voca):
    vec = np.ndarray((len(x_data), len(voca)))
    for i, s in enumerate(tqdm(x_data)): 
        for j, w in enumerate(voca):
            if str(w) in str(s):
                vec[i][j] = 1
            else:
                vec[i][j] = 0
    return sparse.csr_matrix(vec)
#     return vec

In [50]:
train_bone = create_vec_sparse(train_x, words)
train_kai_sparse = create_vec_sparse(train_x, kaiRate)
# test_bone = create_vec_sparse(test_x, words)
# test_kai_sparse = create_vec_sparse(test_x, kaiRate)

100%|██████████| 10000/10000 [00:11<00:00, 835.35it/s]


NameError: name 'test_x' is not defined

In [ ]:
# 벡터를 합친다.
def merge_sparse(a, b):
    return sparse.csr_matrix(np.hstack((a.toarray(), b.toarray())))
def merge_np(a, b):
    return np.concatenate((a, b), axis = 1)


train_with_kai_sprs = merge_sparse(train_bone, train_kai_sparse)
test_with_kai_sprs = merge_sparse(test_bone, test_kai_sparse)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

classifier_tfidf_sprs_1 = MultinomialNB()

classifier_tfidf_sprs_1.fit(train_with_kai_sprs, train_y)

# pred_tfidf_sprs_1 = classifier_tfidf_sprs_1.predict(test_with_kai_sprs).tolist()

print('Accuracy1: %.10f' % accuracy_score(test_y, pred_tfidf_sprs_1))